In [ ]:
%store -r KeyError_list
%store -r tracker 

In [1]:
import pandas as pd
import numpy as np
import json
import os
import re
import time
import sys

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [22]:
# do not run 
KeyError_list = pd.Series()
tracker = 0

In [4]:
list_files = pd.Series(os.listdir('companyfacts/'))
list_sub = pd.Series(os.listdir('submissions/'))

In [23]:
for x, y in enumerate(list_files[:10]):    
    
    df_stock = pd.DataFrame()

    # Block of code opens two json files and then creates AccessionNumber List 
    print(f'{x}   {y}')
    
    with open(f'companyfacts/{y}') as datafile:
        data_comp = json.load(datafile)
        
    with open(f'submissions/{y}') as datafile:
        data_sub = json.load(datafile)
    
    df_submissions = pd.DataFrame(data_sub['filings']['recent'])
    df_10K = df_submissions.loc[df_submissions['form'] == '10-K']

    accn_list = df_10K['accessionNumber']
    accn_list = accn_list.reset_index(drop=True)
    
    
    # Block of code adds AccessionNumbers from additional json submission files 
    CIK_num = y[:-5]
    pattern = re.compile(f'{CIK_num}-submissions')

    for i in list_sub:
        matches = pattern.findall(i)
        
        for match in matches:
            
            with open(f'submissions/{i}') as datafile:
                data_sub = json.load(datafile)
         
            df_submissions = pd.DataFrame(data_sub)
            df_10K = df_submissions.loc[df_submissions['form'] == '10-K']

            accn_additions = df_10K['accessionNumber']
            accn_additions = accn_additions.reset_index(drop=True)
            
            accn_list = pd.concat([accn_list, accn_additions], ignore_index=True)
            
    
    ttt = 0 
    st = time.time()
    
    # Block of code pulls appropriate data from json file and then writes to a csv file
    try: 
        for xx, yy in enumerate(data_comp['facts']['us-gaap']):
        
            df = data_comp['facts']['us-gaap'][yy]['units']

            units = pd.Series(df.keys()).iloc[0]

            df = pd.DataFrame(data_comp['facts']['us-gaap'][yy]['units'][units])
            
            
            for ii in accn_list:
                match = df.loc[df['accn'] == ii]

                match['fn_name'] = yy
                match['cik'] = data_comp['cik']
                
                df_stock = pd.concat([df_stock, match])
                
                ttt += 1
            
            df_stock.to_csv('0-9.csv', mode='a', header=False, index=False)
        
        
        et = time.time()
        elap = et - st 
        
        if ttt != 0:
            print(f'\t\t\t\t\t{elap / ttt * 10000}')
        
        
    except KeyError:
                print(f"\t\t\t\t\t\t\t\t\t\t\tKEYERROR: {y}")
                KeyError_list = pd.concat([KeyError_list, pd.Series(y)])

0   CIK0000874214.json
					31.576213520614886
1   CIK0001820931.json
					88.09743324915569
2   CIK0001271940.json
					18.65659748111759
3   CIK0001563227.json
					38.579890956986425
4   CIK0001473845.json
											KEYERROR: CIK0001473845.json
5   CIK0001401106.json
					27.54734321074052
6   CIK0001489161.json
7   CIK0001470550.json
					41.65054281095892
8   CIK0000744452.json
					50.24406580890351
9   CIK0000884247.json


In [13]:
KeyError_list.reset_index(drop=True, inplace=True)

KeyError_list.to_json('test')

In [16]:
%store KeyError_list
%store tracker

Stored 'df_stock' (DataFrame)
Stored 'KeyError_list' (Series)
